In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, r2_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [2]:
users = pd.read_csv("user_spec.csv")
loans = pd.read_csv("loan_result.csv")

In [3]:
joined = loans.join(users.set_index("application_id"), on="application_id")
joined = joined.dropna(subset = ["loan_limit", "loan_rate"])

In [4]:
# yearly_income 변수를 평균값으로 채워넣습니다.
mean_income = joined.yearly_income.mean()
joined.yearly_income = joined.yearly_income.fillna(mean_income)

In [5]:
# birth_year 변수를 중간값으로 채워넣습니다.
median_birth_year = joined.birth_year.median()
joined.birth_year = joined.birth_year.fillna(median_birth_year)
joined.birth_year = pd.to_datetime(joined.birth_year, format = "%Y")

In [6]:
# personal_rehabilitation, existing_loan 관련 변수는 결측치가 매우 많기 때문에 0으로 채워넣습니다.
joined[[
    "personal_rehabilitation_yn", "personal_rehabilitation_complete_yn", 
    "existing_loan_cnt", "existing_loan_amt"
]] = joined[[
    "personal_rehabilitation_yn", "personal_rehabilitation_complete_yn", 
    "existing_loan_cnt", "existing_loan_amt"
]].fillna(0)

In [7]:
# company_enter_month는 현재 시간으로 채워넣습니다.
joined.company_enter_month = joined.company_enter_month.fillna(202206)
joined.company_enter_month = joined.company_enter_month.astype("str").str[:6]
temp = joined.company_enter_month.str[:6]
temp = pd.to_datetime(temp, format = "%Y%m")
joined.company_enter_month = temp

In [8]:
joined.loanapply_insert_time = pd.to_datetime(joined.loanapply_insert_time)
joined.user_id = joined.user_id.astype(int)

In [9]:
# gender 변수는 전체 평균 비율에 맞추어 랜덤하게 채워넣습니다.
gender_na = joined[joined.gender.isna()]
gender_not_na = joined[~joined.gender.isna()]
total_user_num = len(gender_not_na.user_id.unique())
by_user = gender_not_na.groupby("user_id").max()
num_males = by_user.gender.sum()
prob = num_males / total_user_num
na_user_id_list = gender_na.user_id.unique()
np.random.seed(0)
random_array = np.random.binomial(1, prob, len(na_user_id_list))
random_gender = dict(zip(na_user_id_list, random_array))
na_user_genders = joined.loc[joined.user_id.isin(na_user_id_list)].groupby(joined.user_id).max()
not_na_id = na_user_genders[~na_user_genders.gender.isna()].user_id
not_na_gender = na_user_genders[~na_user_genders.gender.isna()].gender
non_random_gender = dict(zip(not_na_id, not_na_gender))
joined.gender = joined.gender.fillna(joined.user_id.map(non_random_gender))
joined.gender = joined.gender.fillna(joined.user_id.map(random_gender))

In [10]:
# 금액과 관련한 변수는 상용로그로 정규화합니다.
joined['birth_year'] = joined['birth_year'].apply(lambda x: int(str(x)[0:4]))
joined['yearly_income'] = joined['yearly_income'].apply(lambda t: np.log10(1+t))
joined['desired_amount'] = joined['desired_amount'].apply(lambda t: np.log10(1+t))
joined['existing_loan_amt'] = joined['existing_loan_amt'].apply(lambda t: np.log10(1+t))
joined['existing_loan_cnt'] = joined['existing_loan_cnt'].apply(lambda t: int(float(t)))

In [11]:
# credit_score의 결측치는 회귀나무를 사용해 채워넣습니다.
# 사용한 변수는 birth_year, gender, yearly_income, 
# personal_rehabilitation 관련 변수, existing_loan 관련 변수입니다.
credit_tree_joined = pd.concat(
    [
        joined,
        pd.get_dummies(joined.income_type, prefix='income'),
        pd.get_dummies(joined.houseown_type, prefix='house')
    ],
    axis=1
).reset_index(drop=True).drop(
    [
        "application_id", "loanapply_insert_time",
        'insert_time',"bank_id", "product_id",
        'income_type','loan_rate','loan_limit',
        'is_applied','user_id','company_enter_month',
        'purpose','employment_type','houseown_type'
    ],
    axis=1
)

In [12]:
train_credit = credit_tree_joined[credit_tree_joined['credit_score'].notna()]
train_null_credit = credit_tree_joined[credit_tree_joined['credit_score'].isna()]

In [13]:
credit_tree_X = train_credit.drop(['credit_score'], axis=1)
credit_tree_y = train_credit['credit_score']
credit_tree_X_train, credit_tree_X_test, credit_tree_y_train, credit_tree_y_test = train_test_split(
    credit_tree_X, credit_tree_y, test_size=0.3, random_state=44
)
credit_tree_model = DecisionTreeRegressor(random_state=44)
credit_tree_model.fit(credit_tree_X_train, credit_tree_y_train)

DecisionTreeRegressor(random_state=44)

In [14]:
# 회귀를 r2 score로 평가해보니 1에 매우 가깝습니다. 즉 충분히 신뢰할 만합니다.
r2_score(credit_tree_y_test, credit_tree_model.predict(credit_tree_X_test))

0.955760685174084

In [15]:
credit_predict = credit_tree_model.predict(train_null_credit.drop(['credit_score'],axis = 1))
null_credit = joined[joined['credit_score'].isna()]
null_credit['credit_score'] = credit_predict

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_4042/3759801311.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_credit['credit_score'] = credit_predict


In [16]:
null_credit['credit_score']= null_credit['credit_score']//10*10

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_4042/4107112860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_credit['credit_score']= null_credit['credit_score']//10*10


In [17]:
data = pd.concat([joined[joined['credit_score'].notna()],null_credit])

In [18]:
data.insert_time = pd.to_datetime(data.insert_time)

In [19]:
# data에서 필요한 변수만 가져옵니다. 또한 income_type 변수를 One-Hot-Encoding 합니다.
data=data[[
    'loan_limit','loan_rate','credit_score',
    'company_enter_month','desired_amount','income_type',
    'insert_time','user_id','application_id', 'is_applied'
]]
data['earnedincome']=(data['income_type']=='EARNEDINCOME')*1
del data ['income_type']

In [20]:
data = data.sort_values(by=['user_id','insert_time'])

In [21]:
user_array=data['user_id'].values
timestamp_array=data['insert_time'].dt.date.values
application_array=data['application_id'].values
array=np.zeros(len(data))
recent_user=0
recent_application=0
i = 0
store = float(0)

In [22]:
# user별로 이전에 application을 승인했는지 여부를 종합한 파생변수를 만듭니다.
# 이전에 application을 승인한 경우 그 사이 흐른 시간만큼 지수적으로 할인합니다.
for u in user_array:
    if u==recent_user:
        if application_array[i]==recent_application:
            array[i]=store
        else:
            store=0
            for t in time_list:
                if timestamp_array[i]==t:
                    store=store+1
                else:
                    store=store+0.9**((timestamp_array[i]-t).days)
                    
            array[i]=store
            time_list.append(timestamp_array[i])
            recent_application=application_array[i]
    else:
        time_list=[]
        store=0
        array[i]=store
        time_list.append(timestamp_array[i])
        recent_user=user_array[i]
        recent_application=application_array[i]
    i=i+1

In [23]:
data['recent_app_count'] = array

In [85]:
problem1_data=data[data['insert_time']<'2022-06-01']
problem1_data=problem1_data.sort_values(by=['is_applied', 'loan_rate'])
problem1_data=problem1_data.drop_duplicates(['application_id'],keep='last')

In [86]:
# 2022년 05월 16일을 기준으로 학습 데이터, 테스트 데이터를 나누었습니다.
problem1_train_data=problem1_data[problem1_data['insert_time']<'2022-05-16']
problem1_train_label=problem1_train_data['is_applied']
del problem1_train_data['is_applied']
problem1_test_data=problem1_data[problem1_data['insert_time']>='2022-05-16']
problem1_test_label=problem1_test_data['is_applied']
del problem1_test_data['is_applied']

In [87]:
del problem1_train_data["insert_time"]
del problem1_test_data["insert_time"]
del problem1_train_data["user_id"]
del problem1_train_data["application_id"]
del problem1_test_data["user_id"]
del problem1_test_data["application_id"]

In [88]:
# CatBoost를 활용해 1단계 문제 (한도 조회시 승인 여부)를 해결합니다.
problem1_model = CatBoostClassifier(iterations=4,
                           depth=15,
                           learning_rate=0.83,
                           loss_function='Logloss',
                           verbose=True)

In [89]:
problem1_model.fit(problem1_train_data,problem1_train_label)

0:	learn: 0.4371297	total: 277ms	remaining: 832ms
1:	learn: 0.3897184	total: 527ms	remaining: 527ms
2:	learn: 0.3720202	total: 841ms	remaining: 280ms
3:	learn: 0.3651273	total: 1.1s	remaining: 0us


In [90]:
applied_ratio = problem1_data.is_applied.mean()

In [91]:
# f1_score이 0.83입니다.
preds_proba=problem1_model.predict_proba(problem1_test_data)
preds_proba=preds_proba[:,1]
f1_score(problem1_test_label,preds_proba> applied_ratio)

0.8333056731114983

In [92]:
# 각 feature의 중요도를 표시합니다.
problem1_model.get_feature_importance()

array([14.23908267, 38.68345526, 15.62255784,  3.17574216, 13.64622827,
        6.69010945,  7.94282434,  0.        ])

In [95]:
problem1_test_data.loc[:, "predict"] = preds_proba > applied_ratio

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_4042/762248113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  problem1_test_data.loc[:, "predict"] = preds_proba > applied_ratio


In [96]:
# 2단계 문제를 풀기 위해 1단계에서 승인된 것으로 표시된 한도 조회에 대하여 데이터를 불러옵니다.
idx = problem1_test_data.loc[problem1_test_data["predict"]].index
app_yes_id= problem1_data.loc[idx, "application_id"]
predict_app = problem2_data.loc[problem2_data.application_id.isin(app_yes_id)]
predict_app.loc[:, ["sum", "is_enough", "predict"]] = 0

/var/folders/mj/jcmvk9nd451b_mnylnj74qrm0000gn/T/ipykernel_4042/1083243462.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_app.loc[:, ["sum", "is_enough", "predict"]] = 0


In [75]:
problem2_data = data.loc[data.insert_time >= "2022-05-16"]
problem2_data = data.loc[data.insert_time < "2022-06-01"]

In [78]:
# 필요한 feature만 가져옵니다.user의 feature은 사용하지 않습니다.
predict_app = predict_app[[
    "application_id", "loan_limit", "loan_rate", "is_applied", 
    "desired_amount", "sum", "is_enough", "predict"
]]

In [79]:
predict_app = predict_app.sort_values(by = ["application_id", "loan_rate"])

In [80]:
predict_app.iloc[0, 5] = predict_app.iloc[0, 1]

In [81]:
for i in range (1, len(predict_app)):
    predict_app.iloc[i, 5] = predict_app.iloc[i, 1]
    if (predict_app.iloc[i, 0] == predict_app.iloc[i-1, 0]):
        predict_app.iloc[i, 5] += predict_app.iloc[i-1, 5]

In [82]:
predict_app["is_enough"] = predict_app["sum"] > predict_app.desired_amount
predict_app.iloc[0, 7] = 1
predict_app.iloc[1, 7] = 1

In [83]:
# loan_rate 순으로 정렬한 후 desired_amount를 넘을 때까지 대출한다고 가정합니다.
for i in range(2, len(predict_app)):
    if (predict_app.iloc[i, 0] == predict_app.iloc[i-1, 0]):
        if (not predict_app.iloc[i, 6]):
            predict_app.iloc[i, 7] = 1
        elif (not predict_app.iloc[i-1, 6]):
            predict_app.iloc[i, 7] = 1
        elif (not predict_app.iloc[i-2, 6]):
            predict_app.iloc[i, 7] = 1
    else:
        predict_app.iloc[i, 7] = 1

In [84]:
# 0.44의 f1_score입니다.
f1_score(predict_app.is_applied, predict_app.predict)

0.44281519441898737

In [45]:
applied_index = predict_app.loc[predict_app.predict == 1].index

In [46]:
data.loc[:, "predict"] = 0
data.loc[applied_index, "predict"] = 1

In [47]:
test_data = data.loc[data.insert_time >= "2022-05-16"]
test_data = test_data.loc[test_data.insert_time < "2022-06-01"]

In [48]:
# 1, 2단계 문제 총합에 대한 f1_score: 0.45입니다.
f1_score(test_data.is_applied, test_data.predict)

0.4522449618295014